In [1]:
# Display the current working directory
getwd();
# If necessary, change the path below to the directory where the data files are stored.
# "." means current directory. On Windows use a forward slash / instead of the usual \.
workingDir = "breast";
setwd(workingDir);
# Load WGCNA package
library(WGCNA)
allowWGCNAThreads()
# The following setting is important, do not omit.
options(stringsAsFactors = FALSE);

[1] "/home/jovyan/work/phd/cancers"

Loading required package: dynamicTreeCut

Loading required package: fastcluster


Attaching package: ‘fastcluster’


The following object is masked from ‘package:stats’:

    hclust





Attaching package: ‘WGCNA’


The following object is masked from ‘package:stats’:

    cor




Allowing multi-threading with up to 2 threads.


In [2]:
add_legend <- function(...) {
  opar <- par(fig=c(0, 1, 0, 1), oma=c(0, 0, 0, 0), 
    mar=c(0, 0, 0, 0), new=TRUE)
  on.exit(par(opar))
  plot(0, 0, type='n', bty='n', xaxt='n', yaxt='n')
  legend(...)
}

In [3]:
datExpr <- t(read.csv("mainTable.csv", check.names=FALSE, row.names = "0"))
genes <- dimnames(datExpr)[[2]]
samples <- dimnames(datExpr)[[1]]
datExpr <- as.data.frame(apply(datExpr,2, as.numeric)) #numeric
dimnames(datExpr)[[1]] <- samples
allTraits <- read.csv("files.dat")

In [4]:
datExpr[1:5,1:5]

,ENSG00000142615,ENSG00000142959,ENSG00000179751,ENSG00000183773,ENSG00000155850
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
87fcecde-b317-4518-bea1-cc8d14a18107.FPKM.txt.gz,0,0,0,0,2
bbf9a30d-3067-491b-9aea-0cb9bed91969.FPKM.txt.gz,0,0,0,0,2
c909a75f-23e8-4c96-82f2-72e46fdb8155.FPKM.txt.gz,0,0,0,0,1
1c89ff2f-6e0a-4244-952b-1391dba1b815.FPKM.txt.gz,0,0,0,0,2
2a6ae2e8-733f-4349-8eec-d25e2b4abf86.FPKM.txt.gz,0,0,0,0,2


In [ ]:
log2_1 <- function(x){
    log2(x+1)
}
datExpr <- as.data.frame(apply(datExpr,2, log2_1))
dimnames(datExpr)[[1]] <- samples

In [ ]:
datExpr[1:5,1:5]

In [5]:
max(datExpr)

[1] 16

In [6]:
head(allTraits)

,file_name,cases.0.submitter_id,cases.0.demographic.gender,cases.0.demographic.days_to_birth,cases.0.demographic.days_to_death,cases.0.demographic.vital_status,cases.0.diagnoses.0.age_at_diagnosis,cases.0.diagnoses.0.days_to_last_follow_up,cases.0.diagnoses.0.last_known_disease_status,cases.0.diagnoses.0.tumor_stage,⋯,cancer.type,Subtype_mRNA,Subtype_DNAmeth,Subtype_protein,Subtype_miRNA,Subtype_CNA,Subtype_Integrative,Subtype_other,Subtype_Selected,Subtype_Selected_Lum
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,⋯,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,3a3d56a9-e18a-4346-9350-bed393553080.FPKM.txt.gz,TCGA-BH-A0B6,female,-17346.0,unknown,Alive,17346.0,2483.0,not reported,stage i,⋯,BRCA,LumA,unknown,unknown,unknown,unknown,unknown,unknown,BRCA.LumA,BRCA.Lum
2,05eb3daf-d392-4ede-87d6-8f0aef8b6372.FPKM.txt.gz,TCGA-BH-A208,female,-17715.0,1759.0,Dead,17715.0,unknown,not reported,stage iib,⋯,BRCA,Normal,unknown,unknown,unknown,unknown,unknown,unknown,BRCA.Normal,BRCA.Normal
3,58050606-3a53-439b-836b-de391c4c630f.FPKM.txt.gz,TCGA-E9-A22B,female,-26278.0,unknown,Alive,26278.0,1167.0,not reported,stage ia,⋯,BRCA,LumA,unknown,unknown,unknown,unknown,unknown,unknown,BRCA.LumA,BRCA.Lum
4,28973531-fd03-4f86-8cf5-4c6941560c37.FPKM.txt.gz,TCGA-A2-A0EW,female,-19413.0,1884.0,Dead,19413.0,905.0,not reported,stage iiic,⋯,BRCA,LumA,unknown,unknown,unknown,unknown,unknown,unknown,BRCA.LumA,BRCA.Lum
5,0b6603e7-a619-40d0-8af8-ddc7d30ffd7e.FPKM.txt.gz,TCGA-AR-A1AY,female,-23927.0,unknown,Alive,23927.0,1026.0,not reported,stage i,⋯,BRCA,Basal,unknown,unknown,unknown,unknown,unknown,unknown,BRCA.Basal,BRCA.Basal
6,e9422605-b2da-42b0-aa64-a98472834140.FPKM.txt.gz,TCGA-A8-A09I,female,-30711.0,unknown,Alive,30711.0,1371.0,not reported,stage iia,⋯,BRCA,LumB,unknown,unknown,unknown,unknown,unknown,unknown,BRCA.LumB,BRCA.Lum


In [7]:
#labels <- c('cancer.type', 'Subtype_Selected') #<-lung
labels <- c('Subtype_Selected', 'Subtype_Selected_Lum') #<-BRCA
datTraits <- allTraits[,labels]
rownames(datTraits) <- allTraits$file_name #<--CHECK
datTraits <- datTraits[rownames(datExpr),]
head(datTraits)

,Subtype_Selected,Subtype_Selected_Lum
,<chr>,<chr>
87fcecde-b317-4518-bea1-cc8d14a18107.FPKM.txt.gz,BRCA.LumB,BRCA.Lum
bbf9a30d-3067-491b-9aea-0cb9bed91969.FPKM.txt.gz,BRCA.LumA,BRCA.Lum
c909a75f-23e8-4c96-82f2-72e46fdb8155.FPKM.txt.gz,BRCA.Basal,BRCA.Basal
1c89ff2f-6e0a-4244-952b-1391dba1b815.FPKM.txt.gz,BRCA.Normal,BRCA.Normal
2a6ae2e8-733f-4349-8eec-d25e2b4abf86.FPKM.txt.gz,BRCA.LumA,BRCA.Lum
0a2c1866-d6e1-4add-bcb0-b297ba1394ac.FPKM.txt.gz,BRCA.LumB,BRCA.Lum


In [8]:
table( dimnames(datExpr)[[1]]==datTraits$Subtype_Selected)


FALSE 
 1222 

In [9]:
y = datTraits$Subtype_Selected

In [10]:
meanExpressionBySample=apply( datExpr,1,mean, na.rm=T)
NumberMissingBySample=apply( is.na(data.frame(datExpr)),1, sum)

## 1

In [11]:
gsg = goodSamplesGenes(datExpr, verbose = 3);
gsg$allOK

 Flagging genes and samples with too many missing values...
  ..step 1
  ..Excluding 5 genes from the calculation due to too many missing samples or zero variance.
  ..step 2


[1] FALSE

In [12]:
sampleTree = hclust(dist(datExpr), method = "average");
# Plot the sample tree: Open a graphic output window of size 12 by 9 inches
# The user should change the dimensions if the window is too large or too small.
sizeGrWindow(12,9)
svg("tests/low_filters/samplehier.svg")
par(cex = 0.6);
par(mar = c(0,4,2,0))
plot(sampleTree, main = "Sample clustering to detect outliers", sub="", xlab="", cex.lab = 1.5,
cex.axis = 1.5, cex.main = 2)
dev.off()

png 
  2

In [13]:
# Determine cluster under the line
clust = cutreeStatic(sampleTree, cutHeight = 1e5, minSize = 10)
table(clust)

clust
   1 
1222 

In [14]:
# clust 1 contains the samples we want to keep.
keepSamples = (clust==1)
datExpr = datExpr[keepSamples, ]
nGenes = ncol(datExpr)
nSamples = nrow(datExpr)

genes <- dimnames(datExpr)[[2]]
samples <- dimnames(datExpr)[[1]]
datExpr <- as.data.frame(apply(datExpr,2, as.numeric)) #numeric
dimnames(datExpr)[[1]] <- samples

In [15]:
head(datExpr)

,ENSG00000142615,ENSG00000142959,ENSG00000179751,ENSG00000183773,ENSG00000155850,ENSG00000176387,ENSG00000243480,ENSG00000171747,ENSG00000120756,ENSG00000184730,⋯,ENSG00000041982,ENSG00000176894,ENSG00000161267,ENSG00000120915,ENSG00000172456,ENSG00000103257,ENSG00000105173,ENSG00000092758,ENSG00000172179,ENSG00000157613
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
87fcecde-b317-4518-bea1-cc8d14a18107.FPKM.txt.gz,0,0,0,0,2,2,0,0,2,2,⋯,1,3,3,4,1,3,1,0,0,4
bbf9a30d-3067-491b-9aea-0cb9bed91969.FPKM.txt.gz,0,0,0,0,2,2,0,0,1,2,⋯,5,2,1,3,1,1,1,0,0,5
c909a75f-23e8-4c96-82f2-72e46fdb8155.FPKM.txt.gz,0,0,0,0,1,4,0,0,1,2,⋯,3,3,3,2,1,7,4,0,0,2
1c89ff2f-6e0a-4244-952b-1391dba1b815.FPKM.txt.gz,0,0,0,0,2,2,0,0,0,2,⋯,4,3,2,2,2,5,1,4,0,2
2a6ae2e8-733f-4349-8eec-d25e2b4abf86.FPKM.txt.gz,0,0,0,0,2,2,0,0,1,1,⋯,5,3,2,4,1,1,0,0,0,5
0a2c1866-d6e1-4add-bcb0-b297ba1394ac.FPKM.txt.gz,0,0,0,0,2,3,0,0,1,1,⋯,5,3,1,3,1,3,1,1,0,4


In [16]:
rownames(allTraits) <- allTraits$file_name
datTraits <- allTraits[keepSamples,labels]
rownames(datTraits) <- rownames(allTraits[keepSamples,labels])
datTraits <- datTraits[rownames(datExpr),]
head(datTraits)

,Subtype_Selected,Subtype_Selected_Lum
,<chr>,<chr>
87fcecde-b317-4518-bea1-cc8d14a18107.FPKM.txt.gz,BRCA.LumB,BRCA.Lum
bbf9a30d-3067-491b-9aea-0cb9bed91969.FPKM.txt.gz,BRCA.LumA,BRCA.Lum
c909a75f-23e8-4c96-82f2-72e46fdb8155.FPKM.txt.gz,BRCA.Basal,BRCA.Basal
1c89ff2f-6e0a-4244-952b-1391dba1b815.FPKM.txt.gz,BRCA.Normal,BRCA.Normal
2a6ae2e8-733f-4349-8eec-d25e2b4abf86.FPKM.txt.gz,BRCA.LumA,BRCA.Lum
0a2c1866-d6e1-4add-bcb0-b297ba1394ac.FPKM.txt.gz,BRCA.LumB,BRCA.Lum


In [17]:
collectGarbage();

In [18]:
# Re-cluster samples
sampleTree2 = hclust(dist(datExpr), method = "average")
fontsize=.5
# Convert traits to a color representation: white means low, red means high, grey means missing entry
traitColors = labels2colors(datTraits);
# Plot the sample dendrogram and the colors underneath.
pdf("tests/low_filters/samplehiercut.pdf")
plotDendroAndColors(sampleTree2, traitColors,
    groupLabels = labels,
    main = "Sample dendrogram and trait heatmap",
    dendroLabels=FALSE,
    rowText=datTraits,
    rowTextIgnore=labels[[2]]
    )
dev.off()

png 
  2

## 2

In [19]:
# Choose a set of soft-thresholding powers
powers = c(c(1:10), seq(from = 12, to=50, by=2))
# Call the network topology analysis function
sft = pickSoftThreshold(datExpr, powerVector = powers, verbose = 5)
# Plot the results:
sizeGrWindow(9, 5)
pdf("tests/low_filters/scaletopology.pdf")
par(mfrow = c(1,2));
cex1 = 0.9;
# Scale-free topology fit index as a function of the soft-thresholding power
plot(sft$fitIndices[,1], -sign(sft$fitIndices[,3])*sft$fitIndices[,2],
xlab="Soft Threshold (power)",ylab="Scale Free Topology Model Fit,signed R^2",type="n",
main = paste("Scale independence"));
text(sft$fitIndices[,1], -sign(sft$fitIndices[,3])*sft$fitIndices[,2],
labels=powers,cex=cex1,col="red");
# this line corresponds to using an R^2 cut-off of h
abline(h=0.90,col="red")
# Mean connectivity as a function of the soft-thresholding power
plot(sft$fitIndices[,1], sft$fitIndices[,5],
xlab="Soft Threshold (power)",ylab="Mean Connectivity", type="n",
main = paste("Mean connectivity"))
text(sft$fitIndices[,1], sft$fitIndices[,5], labels=powers, cex=cex1,col="red")
dev.off()

pickSoftThreshold: will use block size 992.
 pickSoftThreshold: calculating connectivity for given powers...
   ..working on genes 1 through 992 of 992


Warning message in eval(xpr, envir = envir):
“Some correlations are NA in block 1 : 992 .”
Warning message in eval(xpr, envir = envir):
“Some correlations are NA in block 1 : 992 .”


   Power SFT.R.sq  slope truncated.R.sq  mean.k. median.k. max.k.
1      1    0.634  0.645        0.94300 111.0000  1.07e+02 215.00
2      2    0.728 -0.954        0.95700  26.5000  2.11e+01  90.30
3      3    0.907 -1.340        0.97000   9.4500  5.47e+00  50.00
4      4    0.908 -1.410        0.94200   4.3100  1.73e+00  32.20
5      5    0.948 -1.360        0.96600   2.3100  6.30e-01  22.70
6      6    0.970 -1.320        0.96900   1.4000  2.51e-01  17.10
7      7    0.948 -1.290        0.93300   0.9190  1.02e-01  13.50
8      8    0.953 -1.270        0.94200   0.6460  4.30e-02  11.50
9      9    0.933 -1.290        0.91400   0.4790  1.88e-02  10.50
10    10    0.968 -1.250        0.96100   0.3690  8.32e-03   9.66
11    12    0.987 -1.210        0.98600   0.2420  1.73e-03   8.33
12    14    0.958 -1.180        0.95200   0.1740  3.72e-04   7.33
13    16    0.858 -1.150        0.83700   0.1340  8.90e-05   6.56
14    18    0.242 -1.530        0.02640   0.1080  2.21e-05   5.94
15    20  

png 
  2

In [20]:
#net = blockwiseModules(datExpr, power = 30,
#TOMType = "unsigned", minModuleSize = 5,
#reassignThreshold = 0, mergeCutHeight = 0.2,
#numericLabels = TRUE, pamRespectsDendro = FALSE,
#saveTOMs = TRUE,
#saveTOMFileBase = "wgcna",
#verbose = 3)

net = blockwiseModules(datExpr, power = 30,
TOMType = "unsigned", minModuleSize = 20,
reassignThreshold = 0, mergeCutHeight = 0.8,
numericLabels = TRUE, pamRespectsDendro = FALSE,
saveTOMs = TRUE,
saveTOMFileBase = "wgcna",
verbose = 3)

 Calculating module eigengenes block-wise from all genes
   Flagging genes and samples with too many missing values...
    ..step 1
  ..Excluding 5 genes from the calculation due to too many missing samples or zero variance.
    ..step 2
 ..Working on block 1 .
    TOM calculation: adjacency..
    ..will use 2 parallel threads.
     Fraction of slow calculations: 0.000000
    ..connectivity..
    ..matrix multiplication (system BLAS)..
    ..normalization..
    ..done.
   ..saving TOM for block 1 into file wgcna-block.1.RData
 ....clustering..
 ....detecting modules..
 ....calculating module eigengenes..
 ....checking kME in modules..
 ..merging modules that are too close..
     mergeCloseModules: Merging modules whose distance is less than 0.8
     mergeCloseModules: less than two proper modules.
      ..color levels are 0, 1
      ..there is nothing to merge.
       Calculating new MEs...


In [21]:
# open a graphics window
sizeGrWindow(12, 9)
pdf("tests/low_filters/netdendograms.pdf")
# Convert labels to colors for plotting
mergedColors = labels2colors(net$colors)
# Plot the dendrogram and the module colors underneath
plotDendroAndColors(net$dendrograms[[1]], mergedColors[net$blockGenes[[1]]],
"Module colors",
dendroLabels = FALSE, hang = 0.03,
addGuide = TRUE, guideHang = 0.05)
dev.off()

png 
  2

In [22]:
moduleLabels = net$colors
moduleColors = labels2colors(net$colors)
MEs = net$MEs;
geneTree = net$dendrograms[[1]];

## 3

In [ ]:
# Lung
datTraits$cancer.type <- match(datTraits$cancer.type, unique(datTraits$cancer.type))
datTraits$Subtype_Selected <- match(datTraits$Subtype_Selected, unique(datTraits$Subtype_Selected))

In [23]:
# BRCA
datTraits$Subtype_Selected <- match(datTraits$Subtype_Selected, unique(datTraits$Subtype_Selected))
datTraits$Subtype_Selected_Lum <- match(datTraits$Subtype_Selected_Lum, unique(datTraits$Subtype_Selected_Lum))

In [24]:
# Define numbers of genes and samples
nGenes = ncol(datExpr);
nSamples = nrow(datExpr);
# Recalculate MEs with color labels
MEs0 = moduleEigengenes(datExpr, moduleColors)$eigengenes
MEs = orderMEs(MEs0)
moduleTraitCor = cor(MEs, datTraits, use = "p");
moduleTraitPvalue = corPvalueStudent(moduleTraitCor, nSamples);

In [25]:
pdf("tests/low_filters/moduleTrait.pdf", width=9, height=18)
# Will display correlations and their p-values
textMatrix = paste(signif(moduleTraitCor, 2), "\n(",
signif(moduleTraitPvalue, 1), ")", sep = "");
dim(textMatrix) = dim(moduleTraitCor)
par(mar = c(6, 8.5, 3, 3));
# Display the correlation values within a heatmap plot
labeledHeatmap(Matrix = moduleTraitCor,
xLabels = labels,
yLabels = names(MEs),
ySymbols = names(MEs),
colorLabels = FALSE,
colors = blueWhiteRed(50),
textMatrix = textMatrix,
setStdMargins = FALSE,
cex.text = 0.5,
zlim = c(-1,1),
main = paste("Module-trait relationships"))
dev.off()

png 
  2

In [26]:
# Define variable weight containing the weight column of datTrait
weight = as.data.frame(rownames(datTraits));
names(weight) = "weight"
# names (colors) of the modules
modNames = substring(names(MEs), 3)
geneModuleMembership = as.data.frame(cor(datExpr, MEs, use = "p"));
MMPvalue = as.data.frame(corPvalueStudent(as.matrix(geneModuleMembership), nSamples));
names(geneModuleMembership) = paste("MM", modNames, sep="");
names(MMPvalue) = paste("p.MM", modNames, sep="");
geneTraitSignificance = as.data.frame(cor(datExpr, weight, use = "p"));
GSPvalue = as.data.frame(corPvalueStudent(as.matrix(geneTraitSignificance), nSamples));
names(geneTraitSignificance) = paste("GS.", names(weight), sep="");
names(GSPvalue) = paste("p.GS.", names(weight), sep="");

Warning message in storage.mode(y) <- "double":
“NAs introduced by coercion”


In [27]:
modNames

[1] "turquoise" "grey"

In [28]:
module = "grey"
column = match(module, modNames);
moduleGenes = moduleColors==module;
sizeGrWindow(7, 7);
pdf("tests/low_filters/modulemembership.pdf")
par(mfrow = c(1,1));
verboseScatterplot(abs(geneModuleMembership[moduleGenes, column]),
    abs(geneTraitSignificance[moduleGenes, 1]),
    xlab = paste("Module Membership in", module, "module"),
    ylab = "Gene significance for primary_site",
    main = paste("Module membership vs. gene significance\n"),
    cex.main = 1.2, cex.lab = 1.2, cex.axis = 1.2, col = module)
dev.off()

Warning message in sqrt(nSamples - 2):
“NaNs produced”
Warning message in min(x):
“no non-missing arguments to min; returning Inf”
Warning message in max(x):
“no non-missing arguments to max; returning -Inf”


ERROR: Error in plot.window(...): need finite 'ylim' values


## 5 

In [29]:
# Re-cluster samples
sampleTree3 = hclust(dist(MEs0), method = "average")
# Convert traits to a color representation: white means low, red means high, grey means missing entry
traitColors = labels2colors(datTraits);
# Plot the sample dendrogram and the colors underneath.
pdf("tests/low_filters/nethier.pdf", width=10)
# Set the plot margin: bottom, left, top & right
par(mar = c(5, 4, 1.4, 0.2) + 0.1,
    xpd = NA) # allow content to go into outer margin 
plotDendroAndColors(sampleTree3, traitColors,
groupLabels = labels,
main = "Network dendrogram and trait heatmap",
dendroLabels=FALSE,
#rowText=allTraits[keepSamples,labels],
#rowTextIgnore=labels[[2]],
addGuide=TRUE)
dev.off()

png 
  2

In [30]:
rownames(allTraits)=allTraits$file_name
unique(allTraits[rownames(datTraits),'Subtype_Selected'])

[1] "BRCA.LumB"   "BRCA.LumA"   "BRCA.Basal"  "BRCA.Normal" "unknown"    
[6] "BRCA.Her2"

In [31]:
svg("tests/low_filters/nethier_legend.svg", width=10)
add_legend("topleft", legend = unique(allTraits[rownames(datTraits),'Subtype_Selected']), pch = 1, pt.cex = 1, cex = 1.5, bty = 'n', ncol=2,
       inset = c(0.1, 0.), # place outside
       title = "SMTS", 
       col = traitColors)
dev.off()

Warning message in par(fig = c(0, 1, 0, 1), oma = c(0, 0, 0, 0), mar = c(0, 0, 0, :
“calling par(new=TRUE) with no plot”


png 
  2

In [32]:
collectGarbage()

In [33]:
clust = cutreeStatic(sampleTree3, cutHeight = 0.9, minSize = 10)
table(clust)

clust
   1 
1222 

In [34]:
# Re-cluster samples after cut
sampleTree4 = hclust(dist(MEs0[(clust==1),]), method = "average")
# Convert traits to a color representation: white means low, red means high, grey means missing entry
traitColors = labels2colors(datTraits[(clust==1),]);
# Plot the sample dendrogram and the colors underneath.
pdf("tests/low_filters/nethiercut.pdf")
plotDendroAndColors(sampleTree4, traitColors,
groupLabels = labels,
main = "Network dendrogram and trait heatmap",
dendroLabels=FALSE)
dev.off()

png 
  2

In [ ]:
# Calculate topological overlap anew: this could be done more efficiently by saving the TOM
# calculated during module detection, but let us do it again here.
dissTOM = 1-TOMsimilarityFromExpr(datExpr, power = 6);
# Transform dissTOM with a power to make moderately strong connections more visible in the heatmap
plotTOM = dissTOM^7;
# Set diagonal to NA for a nicer plot
diag(plotTOM) = NA;
# Call the plot function
sizeGrWindow(9,9)
svg("networkmap.svg")
TOMplot(plotTOM, geneTree, moduleColors, main = "Network heatmap plot, all genes")
dev.off()

In [ ]:
#selct genes to be faster
nSelect = nGenes*0.7
# For reproducibility, we set the random seed
set.seed(10);
select = sample(nGenes, size = nSelect);
selectTOM = dissTOM[select, select];
# There’s no simple way of restricting a clustering tree to a subset of genes, so we must re-cluster.
selectTree = hclust(as.dist(selectTOM), method = "average")
selectColors = moduleColors[select];
# Open a graphical window
sizeGrWindow(9,9)
svg("networkmap.svg")
# Taking the dissimilarity to a power, say 10, makes the plot more informative by effectively changing
# the color palette; setting the diagonal to NA also improves the clarity of the plot
plotDiss = selectTOM^7;
diag(plotDiss) = NA;
TOMplot(plotDiss, selectTree, selectColors, main = "Network heatmap plot, selected genes")
dev.off()

In [ ]:
# Recalculate module eigengenes
MEs = moduleEigengenes(datExpr, moduleColors)$eigengenes
# Isolate weight from the clinical traits
weight = as.data.frame(datTraits$SMTS);
names(weight) = "SMTS"
# Add the weight to existing module eigengenes
MET = orderMEs(cbind(MEs, weight))
# Plot the relationships among the eigengenes and the trait
sizeGrWindow(5,7.5);
svg("eigenHeat.svg")
par(cex = 0.9)
plotEigengeneNetworks(MET, "", marDendro = c(0,4,1,2), marHeatmap = c(3,4,1,2), cex.lab = 0.8, xLabelsAngle = 90)
dev.off()

In [ ]:
# Plot the dendrogram
sizeGrWindow(6,6);
svg("eigendendogram.svg")
par(cex = 1.0)
plotEigengeneNetworks(MET, "Eigengene dendrogram", marDendro = c(0,4,2,0),
plotHeatmaps = FALSE)
# Plot the heatmap matrix (note: this plot will overwrite the dendrogram plot)
par(cex = 1.0)
plotEigengeneNetworks(MET, "Eigengene adjacency heatmap", marHeatmap = c(3,4,2,2),
plotDendrograms = FALSE, xLabelsAngle = 90)
dev.off()

Values of *k* must be read from *clustersizes.txt file*

In [35]:
write.csv(cutree(sampleTree4, k=8), "tests/low_filters/wgcna/wgcna_level_3_labels.csv")
write.csv(cutree(sampleTree4, k=29), "tests/low_filters/wgcna/wgcna_level_2_labels.csv")
write.csv(cutree(sampleTree4, k=149), "tests/low_filters/wgcna/wgcna_level_1_labels.csv")
write.csv(cutree(sampleTree4, k=1204), "tests/low_filters/wgcna/wgcna_level_0_labels.csv")

In [36]:
write.csv(geneModuleMembership, "tests/low_filters/wgcna/wgcna_level_0_word-dist.csv")

In [37]:
write.csv(MEs, "tests/low_filters/wgcna/wgcna_level_0_topic-dist.csv")